In [2]:
# imports
from transforms import AddChannel, Iso, PadSz

# Utilities
import os
import time
import pickle
from pathlib import Path

# Fastai
from fastai import *
from fastai.torch_basics import *
from fastai.basics import *

# PyTorch
from torchvision.models.video import r3d_18
from fastai.callback.all import SaveModelCallback
from torch import nn

# 3D extension to FastAI
# from faimed3d.all import *

# Input IO
import SimpleITK as sitk
import meshio

# Numpy and Pandas
import numpy as np
import pandas as pd
from pandas import DataFrame as DF

# Helper functions
from helpers.preprocess import get_data_dict, paths2objs, folder2objs, seg2mask, mask2bbox, print_bbox, get_bbox_size, print_bbox_size
from helpers.general import sitk2np, np2sitk, print_sitk_info, round_tuple, lrange, lmap, get_roi_range, numbers2groups
from helpers.viz import viz_axis

In [3]:
# Get path to my data on 4 TB HD
hd        = "/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata"
train_src = hd + "/samir_labels"

# print
print("Folders in train path: ", end=""); print(*os.listdir(train_src), sep=", ")

# get data
data = {}
folders = os.listdir(train_src)
for folder in folders: data.update(get_data_dict(f"{train_src}/{folder}"))

# all items
items = list(data.values())


# Create (MR path, Segm path) item from MR path
def get_folder_name(s):
    start = s.index("samir_labels/")
    s = s[start + len("samir_labels/50373-50453/"):]
    return s[0:s.index("/")]

# print
print(f"Total {len(items)} items in dataset.")

Folders in train path: 50155-50212, 50313-50372, 50213-50312, 50373-50453, 50002-50153
Total 335 items in dataset.


In [16]:
segm_path = items[0][1]
fn = f"{str(Path(segm_path).parent)}/seg.pt"
print(fn)
a = torch.load(f"{str(Path(segm_path).parent)}/seg.pt")
n_total = torch.numel(a)
n_segm = torch.count_nonzero(a)
print(float(n_segm)/n_total)

/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50201/seg.pt
0.0014727392862009447


In [14]:
torch.numel(torch.rand((2,3,4)))

24

In [13]:
torch.bincount(a)

RuntimeError: "bincount_cpu" not implemented for 'Bool'

In [17]:
start = time.time()
n_total = 0
n_segm  = 0

for _,segm_path in items:
    a = torch.load(f"{str(Path(segm_path).parent)}/seg.pt")
    n_total += torch.numel(a)
    n_segm  += torch.count_nonzero(a)
        
class_weights = 1.0 * n_segm/n_total
torch.save(class_weights, "saved_metadata/class_weights.pt")

print(class_weights)

elapsed = time.time() - start
print(f"Elapsed: {elapsed} s for {len(items)} items.")

tensor(0.0019)


NameError: name 'unique' is not defined

In [18]:
class_weights = torch.load("saved_metadata/class_weights.pt")

In [19]:
class_weights

tensor(0.0019)